# Прогнозирование оттока клиентов

## Отчет о проведенном исследовании

## Цели и задачи
**Решается задача прогнозирования оттока клиентов, особенно актуальна для сервис провайдеров с распространением услуги близким к 100%. Данные взяты из клиентской базы французской телекоммуникационной компанией Orange, анонимизированы и обфусцированы. Состоит из 50тыс объектов и включает 230 переменных, 190 – числовые, остальные 40 – категориальные.**

In [1]:
# изначальный вид данных
import pandas as pd
data = pd.read_csv("orange_small_churn_train_data.csv", index_col='ID')
data.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230,labels
ID,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,3052.0,NaN,NaN,NaN,NaN,...,vr93T2a,LM8l689qOp,NaN,NaN,fKCe,02N6s8f,xwM2aC7IdeMC0,NaN,NaN,-1
1,NaN,NaN,NaN,NaN,NaN,1813.0,7.0,NaN,NaN,NaN,...,6hQ9lNX,LM8l689qOp,NaN,ELof,xb3V,RAYp,55YFVY9,mj86,NaN,-1
2,NaN,NaN,NaN,NaN,NaN,1953.0,7.0,NaN,NaN,NaN,...,catzS2D,LM8l689qOp,NaN,NaN,FSa2,ZI9m,ib5G6X1eUxUn6,mj86,NaN,-1
3,NaN,NaN,NaN,NaN,NaN,1533.0,7.0,NaN,NaN,NaN,...,e4lqvY0,LM8l689qOp,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN,1
4,NaN,NaN,NaN,NaN,NaN,686.0,7.0,NaN,NaN,NaN,...,MAz3HNj,LM8l689qOp,NaN,NaN,WqMG,RAYp,F2FyR07IdsN7I,NaN,NaN,-1


## Решенные в рамках проекта задачи:
* проведение описательного анализа данных;
* формирование ключевых метрик оценки качества модели;
* построение baseline-решения;
* эксперименты с различной обработкой входных данных;
* построение модели прогнозирования и ее оптимизация;
* оценка потенциального экономического эффекта от внедрения финальной модели прогнозирования оттока.

## Применение
** Для того, чтобы получить прогноз, необходимо подготовить данные согласно алгоритму ниже и применить к ним обученный классификатор следующим образом:**

In [ ]:
result = clf.predict_proba(test)
churn_probabilities = [x[1] for x in result]

** Получаем массив вероятностей ухода пользователя, откуда можно установить, какие пользователи наиболее склонны к оттоку ** 

## Измерение качества и критерий успеха
** Данные сильно несбалансированы, так как клиентов, которые собираются уходить гораздо меньше, чем тех, кто остается. Поэтому предлагается использовать метрику AUC-PRC (площадь под кривой точность-полнота), так как данная метрика устойчива к дисбалансу классов. Для повышения точности оценки и защиты от переобучения использовалась стандартная техника кросс-валидации.
А также замечу, что в дополнение к AUC-PRC в качестве контрольной метрики использовалась AUC-ROC, как наиболее популярная метрика и применимая к широкому кругу задач.**

** Если не решать задачу прогнозирования оттока, то для произвольного клиента мы можем предположить, что он уйдет с вероятностью 50%. Таким образом, минимальным критерием успеха для нашей модели прогнозирования будет качество выше 50%. Но, естественно, чем выше качество классификации, тем лучше. **

## Описание развития и подготовки модели.
### Анализ данных
** В ходе первичного анализа данных было выявлено, что данные достаточно шумные и корреляция с целевой переменно зависит в основном от дисперсии данных. **

![title](correlation.png)

** Наиболее коррелирующие признаки следующие:
Var153, Var38, Var133, Var76, Var217, Var214, Var200
Они предположительно вносят наибольший вклад. **

### Выбор классификатора
** При построеннии бейзлайн решения был произведен выбор метрики и подбор классификатора. **
1\. Ridge классификатор

![title](ridge.png)

2\. Логистическая регрессия

![title](logistic_regr.png)

3\. Случайный лес

![title](random_forest.png)

4\. Градиентный бустинг

![title](grad_boost.png)

** Градиентный бустинг был выбран в качестве основного классификатора. **


### Подбор параметров
** В ходе работы было проверено множество наборов параметров с помощью GridSearchCV, например **

parameters_grid = {

    'max_depth' : [5, 10, 15, 20, 25],
    'min_samples_leaf' : [1, 2, 3, 5, 8],
    'n_estimators' : [10, 50, 100, 300],
    'criterion' : ['gini', 'entropy'],
    'max_features' : [0.2, 0.4, 0.6, 0.7, 0.8, 0.9]

}

** Также производился подбор весов классов **

params = {

    "class_weight": [{1: 4}, {1: 8}, {1: 10}, {1: 15}, {1: 20}, {1: 50}]

}


** Для градиентного бустинга оптимальный набор параметров: **

n_estimators=300, learning_rate=0.2, max_depth=5, loss='exponential', max_features=0.3

** Однако, проверка на отложенной выборке показала, что бустинг со стандартными параметрами и n_estimators=200
дает все-таки более хорошее качество, так как, видимо, слабее переобучен. **

## Построение конечной модели
1\. ** Процесс подготовки данных можно проиллюстрировать следующим образом: **

![title](data_preparing.jpg)

** После преобработки данные выглядят гораздо лучше!**

In [4]:
prepared = pd.read_csv("data_prepared.csv", index_col='Id')
prepared.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var219,Var220,Var221,Var222,Var223,Var225,Var226,Var227,Var228,Var229
Id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.023163,0.00,0.0,0.0,0.0,...,0.52381,0.729371,0.000000,0.948019,0.0,0.333333,0.636364,0.000000,1.000000,0.333333
1,0.0,0.0,0.0,0.0,0.0,0.013760,0.05,0.0,0.0,0.0,...,0.52381,0.731469,0.666667,0.088345,0.0,0.333333,1.000000,0.333333,0.068966,0.333333
2,0.0,0.0,0.0,0.0,0.0,0.014822,0.05,0.0,0.0,0.0,...,0.52381,0.081585,1.000000,0.641259,0.0,0.333333,0.318182,0.500000,0.862069,0.333333
3,0.0,0.0,0.0,0.0,0.0,0.011635,0.05,0.0,0.0,0.0,...,0.52381,0.986480,0.666667,0.667599,0.0,0.333333,1.000000,0.333333,0.275862,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.005206,0.05,0.0,0.0,0.0,...,0.52381,0.418881,0.666667,0.351049,0.0,0.333333,0.545455,0.333333,0.275862,0.000000


2\. ** Далее необходимо обучить на подготовленных данных GradientBoostingClassifier и модель готова к работе! **

## Оценка результатов
** Данная модель имеет уже неплохое качество, так как обучение происходило на достаточно большом массиве данных.
На графике кривой обучения можно заметить, что дальнее увеличение датасета не даст значительного прироста качества. **

![title](learning_curve.png)

** Также отметим, что оценка качества с помощью кросс-валидации и метрики качества AUC-PRC позволяет утверждать, что модель не переобучена и будет хорошо себя проявлять на новых данных. **

### Экономический эффект
** Клиентская база orange насчитывает более 200млн пользователей. По статистике, уходит примерно 7% пользователей в год. Если взять за основу, что средний клиент тратит на услуги 10 евро в месяц, получаем потенциальную потерю прибыли _~140млн_ евро в месяц! **

**Построенная модель имеет качество в 73%, что позволяет определить группу пользователей склонных к оттоку, размером не более 10% от общего числа клиентов, содержащую наши 7% действительно собирающихся уходить пользоватей с точностью 95%. **

** Таким образом, считая, что пользователи примут наше предложение всего лишь с вероятностью 50% и применяя методики удержания с бюджетом 2 евро на человека только к выделенной группе, получаем дополнительную прибыль в _~30млн_ евро! **


## Итоги
** Результатом проделанной работы является обученный на подготовленных данных классификатор, готовый к практическому применению, алгоритм подготовки данных для классификации, а также оценки качества и эффекта от применения. **
### Улучшение модели
** Для дальнейшего развития и улучшения модели можно собрать еще данных, а также дальше поэкспериментировать с новыми моделями и преобразованиями признаков.**

### Внедрение
** Для внедрения классификатора потребуется дополнительно автоматизировать процесс предобработки новых данных согласно описанному алгоритму. **

### Тестирование
** При оценке экономического эффекта была предложена конкретная вероятность принятия предложения. Но перед внедрением в продакшен необходимо этот параметр уточнить с помощью A/Б тестирования **